In [1]:
import imaplib
import email
from email.header import decode_header
from dotenv import load_dotenv
import os
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# CONFIGURATION
IMAP_SERVER = 'imap.web.de'
EMAIL_ACCOUNT = os.getenv('EMAIL_ACCOUNT')
PASSWORD = os.getenv('EMAIL_PASSWORD')

today = datetime.today().strftime("%d-%b-%Y")

In [18]:
# DECODE HEADERS
def decode_mime_words(header_val):
    decoded = decode_header(header_val)
    return ''.join([
        str(t[0], t[1] or 'utf-8') if isinstance(t[0], bytes) else str(t[0])
        for t in decoded
    ])

# READ EMAILS
def read_emails(mail):

    mail.select('INBOX')
    status, messages = mail.search(None, 'ON', '28-May-2025')
    email_ids = messages[0].split()

    full_emails = []

    for num in email_ids[::-1]:
        status, data = mail.fetch(num, '(RFC822)')
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)

        subject = decode_mime_words(msg.get("Subject", ""))
        from_ = decode_mime_words(msg.get("From", ""))
        date_ = msg.get("Date", "")

        # Extract body
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain" and part.get_content_disposition() is None:
                    body = part.get_payload(decode=True).decode(errors='ignore')
                    break
        else:
            body = msg.get_payload(decode=True).decode(errors='ignore')

        # Combine all parts into a single string
        full_email = (
            f"From: {from_}\n"
            f"Subject: {subject}\n"
            f"Date: {date_}\n"
            f"{'-'*50}\n"
            f"{body.strip()}\n"
        )

        full_emails.append(full_email)

    return ' '.join(full_emails)

In [26]:
# CONNECT TO SERVER
def connect():
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL_ACCOUNT, PASSWORD)
    return mail

# DECODE HEADERS
def decode_mime_words(header_val):
    decoded = decode_header(header_val)
    return ''.join([
        str(t[0], t[1] or 'utf-8') if isinstance(t[0], bytes) else str(t[0])
        for t in decoded
    ])

# READ EMAILS
def read_emails(mail):
    mail.select('INBOX')
    status, messages = mail.search(None, 'ON', '28-May-2025') # just from today
    email_ids = messages[0].split()

    for num in email_ids[::-1]:
        status, data = mail.fetch(num, '(RFC822)')
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)

        subject = decode_mime_words(msg.get("Subject", ""))
        from_ = decode_mime_words(msg.get("From", ""))
        date_ = msg.get("Date", "")

        print("="*50)
        print(f"From: {from_}")
        print(f"Subject: {subject}")
        print(f"Date: {date_}")

        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain" and part.get_content_disposition() is None:
                    body = part.get_payload(decode=True).decode(errors='ignore')
                    print("\nBody:\n", body[:500], '...\n')
                    break
        else:
            body = msg.get_payload(decode=True).decode(errors='ignore')
            print("\nBody:\n", body[:500], '...\n')

# MAIN
if __name__ == '__main__':
    try:
        mail = connect()
        read_emails(mail)
        mail.logout()
    except Exception as e:
        print(f"Error: {e}")

Error: Unknown IMAP4 command: 'message_from_bytes'
